Code based on https://github.com/pytorch/examples/blob/master/mnist/main.py

In this exercise, we are going to implement a [ResNet-like](https://arxiv.org/pdf/1512.03385.pdf) architecture for the image classification task.
The model is trained on the [MNIST](https://en.wikipedia.org/wiki/MNIST_database) dataset.

Tasks:

    1. Implement residual connections in the missing places in the code.

    2. Check that the given implementation reaches 97% test accuracy after a few epochs.

    3. Check that when extending the residual blocks to 20 (having 40+ layers total), the model still trains well, i.e., achieves 97+% accuracy after three epochs.

In [33]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms


In [34]:
class ResidualConnection(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(
                in_channels=in_channels,
                out_channels=out_channels,
                kernel_size=3,
                padding=1,
            ),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )
        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(
                in_channels=out_channels,
                out_channels=out_channels,
                kernel_size=3,
                padding=1,
            ),
            nn.BatchNorm2d(out_channels),
        )
        self.in_channels = in_channels
        self.out_channels = out_channels

    def forward(self, x):
        # TODO: implement forward pass.
        fx = self.conv_block_1(x)
        fx = self.conv_block_2(fx)
        if self.in_channels == self.out_channels:
            x_prime = x
        elif self.in_channels < self.out_channels:
            padding = torch.zeros(x.shape[2], x.shape[3]).unsqueeze(0).unsqueeze(0)
            padding_tensor = padding.repeat(x.shape[0], self.out_channels - self.in_channels, 1, 1)
            x_prime = torch.cat((x, padding_tensor), dim=1)
        return nn.functional.relu(torch.add(fx, x_prime))

In [35]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.rc = nn.Sequential(
            ResidualConnection(1, 16),
            ResidualConnection(16, 16),
            ResidualConnection(16, 16),
        )
        self.fc = nn.Linear(
            28 * 28 * 16, 10
        )  # 28 * 28 * 16 is the size of flattened output of the last ResidualConnection

    def forward(self, x):
        x = self.rc(x)
        x = nn.Flatten(start_dim=1)(x)
        x = self.fc(x)
        output = nn.LogSoftmax(dim=1)(x)
        return output


In [36]:
def train(model, device, train_loader, optimizer, epoch, log_interval):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch,
                    batch_idx * len(data),
                    len(train_loader.dataset),
                    100.0 * batch_idx / len(train_loader),
                    loss.item(),
                )
            )


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(
                output, target, reduction="sum"
            ).item()  # sum up batch loss
            pred = output.argmax(
                dim=1, keepdim=True
            )  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print(
        "\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
            test_loss,
            correct,
            len(test_loader.dataset),
            100.0 * correct / len(test_loader.dataset),
        )
    )


In [37]:
batch_size = 256
test_batch_size = 1000
epochs = 3
lr = 1e-2
use_cuda = False
seed = 1
log_interval = 10


In [38]:
use_cuda = not use_cuda and torch.cuda.is_available()

torch.manual_seed(seed)
device = torch.device("cuda" if use_cuda else "cpu")

train_kwargs = {"batch_size": batch_size}
test_kwargs = {"batch_size": test_batch_size}
if use_cuda:
    cuda_kwargs = {"num_workers": 1, "pin_memory": True, "shuffle": True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)


In [39]:
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
)
dataset1 = datasets.MNIST("../data", train=True, download=True, transform=transform)
dataset2 = datasets.MNIST("../data", train=False, transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)


In [40]:
model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch, log_interval)
    test(model, device, test_loader)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.526710
Train Epoch: 1 [2560/60000 (4%)]	Loss: 14.373219
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.369129
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.514152
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.605485
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.821124
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.372500
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.310326
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.356186
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.220788
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.364109
Train Epoch: 1 [28160/60000 (47%)]	Loss: 0.325376
Train Epoch: 1 [30720/60000 (51%)]	Loss: 0.199410
Train Epoch: 1 [33280/60000 (55%)]	Loss: 0.170073
Train Epoch: 1 [35840/60000 (60%)]	Loss: 0.244049
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.174176
Train Epoch: 1 [40960/60000 (68%)]	Loss: 0.142816
Train Epoch: 1 [43520/60000 (72%)]	Loss: 0.118412
Train Epoch: 1 [46080/60000 (77%)]	Loss: 0.325115
Train Epoch: 1 [48640/60000 (81%)]	Loss: 0.076910
Train Epo

In [46]:
class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        self.rc = nn.Sequential(
            ResidualConnection(1, 16),
            ResidualConnection(16, 16),
            ResidualConnection(16, 16),
            ResidualConnection(16, 16),
            ResidualConnection(16, 16),
            
            ResidualConnection(16, 16),
            ResidualConnection(16, 16),
            ResidualConnection(16, 16),
            ResidualConnection(16, 16),
            ResidualConnection(16, 16),
            
            ResidualConnection(16, 16),
            ResidualConnection(16, 16),
            ResidualConnection(16, 16),
            ResidualConnection(16, 16),
            ResidualConnection(16, 16),
            
            ResidualConnection(16, 16),
            ResidualConnection(16, 16),
            ResidualConnection(16, 16),
            ResidualConnection(16, 16),
            ResidualConnection(16, 16),
        )
        self.fc = nn.Linear(
            28 * 28 * 16, 10
        )  # 28 * 28 * 16 is the size of flattened output of the last ResidualConnection

    def forward(self, x):
        x = self.rc(x)
        x = nn.Flatten(start_dim=1)(x)
        x = self.fc(x)
        output = nn.LogSoftmax(dim=1)(x)
        return output

In [47]:
model = Net2().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

for epoch in range(1):
    train(model, device, train_loader, optimizer, epoch, log_interval)
    test(model, device, test_loader)

Train Epoch: 0 [0/60000 (0%)]	Loss: 4.266386
Train Epoch: 0 [2560/60000 (4%)]	Loss: 36.106873
Train Epoch: 0 [5120/60000 (9%)]	Loss: 4.755883
Train Epoch: 0 [7680/60000 (13%)]	Loss: 5.631119
Train Epoch: 0 [10240/60000 (17%)]	Loss: 3.658088
Train Epoch: 0 [12800/60000 (21%)]	Loss: 2.227066
Train Epoch: 0 [15360/60000 (26%)]	Loss: 2.833503
Train Epoch: 0 [17920/60000 (30%)]	Loss: 2.367362
Train Epoch: 0 [20480/60000 (34%)]	Loss: 1.846981


KeyboardInterrupt: 